# Project 3: Build Decision Tree for Bank Loan Modelling

    DV - "Personal Loan"

    IDV - Output of RF Algorithm

In [1]:
import numpy as np
import pandas as pd
from sklearn import tree
from sklearn import preprocessing

In [4]:
df = pd.read_excel("Bank.xlsx","Data")

In [5]:
df.head(10)

,ID,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
0,1,25,1,49,91107,4,1.6,1,0,0,1,0,0,0
1,2,45,19,34,90089,3,1.5,1,0,0,1,0,0,0
2,3,39,15,11,94720,1,1.0,1,0,0,0,0,0,0
3,4,35,9,100,94112,1,2.7,2,0,0,0,0,0,0
4,5,35,8,45,91330,4,1.0,2,0,0,0,0,0,1
5,6,37,13,29,92121,4,0.4,2,155,0,0,0,1,0
6,7,53,27,72,91711,2,1.5,2,0,0,0,0,1,0
7,8,50,24,22,93943,1,0.3,3,0,0,0,0,0,1
8,9,35,10,81,90089,3,0.6,2,104,0,0,0,1,0
9,10,34,9,180,93023,1,8.9,3,0,1,0,0,0,0


In [6]:
df.columns

Index(['ID', 'Age', 'Experience', 'Income', 'ZIP Code', 'Family', 'CCAvg',
       'Education', 'Mortgage', 'Personal Loan', 'Securities Account',
       'CD Account', 'Online', 'CreditCard'],
      dtype='object')

# Random Forest for selecting important features

In [7]:
from sklearn.ensemble import RandomForestClassifier

In [8]:
df.isna().sum()

ID                    0
Age                   0
Experience            0
Income                0
ZIP Code              0
Family                0
CCAvg                 0
Education             0
Mortgage              0
Personal Loan         0
Securities Account    0
CD Account            0
Online                0
CreditCard            0
dtype: int64

There are no null values.

In [9]:
objList = df.select_dtypes(include = "object").columns
print (objList)

Index([], dtype='object')


All variables are numerical

In [10]:
rf_model=RandomForestClassifier(n_estimators=1000,max_features=2,oob_score=True)

In [11]:
features=['ID', 'Age', 'Experience', 'Income', 'ZIP Code', 'Family', 'CCAvg',
       'Education', 'Mortgage', 'Securities Account',
       'CD Account', 'Online', 'CreditCard']

In [12]:
rf_model.fit(X=df[features],y=df['Personal Loan'])

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features=2,
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=1000,
                       n_jobs=None, oob_score=True, random_state=None,
                       verbose=0, warm_start=False)

In [13]:
print(rf_model.oob_score_)

0.9856


Thus, the model is 98.56 % accurate.

In [14]:
for feature,imp in zip(features,rf_model.feature_importances_):
    print(feature,imp)

ID 0.043245501731363385
Age 0.03833606799819292
Experience 0.037317367381929284
Income 0.32575260472561013
ZIP Code 0.04058947472461188
Family 0.08485162145183583
CCAvg 0.16871218201757102
Education 0.14373715926022038
Mortgage 0.04185240488104037
Securities Account 0.00521812501762367
CD Account 0.05320102292217382
Online 0.007865260413529343
CreditCard 0.00932120747429807


The imp features in this dataset can be: Income,Family,CCAvg, Education.

In [15]:
imp_features=['Income','Family','CCAvg','Education']
rf_model.fit(X=df[imp_features],y=df['Personal Loan'])
print(rf_model.oob_score_)

0.9876


This model is now 98.76% accurate.

# Classification

In [17]:
tree_model = tree.DecisionTreeClassifier(max_depth=8)
features=['Income','Family','CCAvg','Education']
tree_model.fit(X=df[features],y=df['Personal Loan'])

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=8, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [18]:
with open('loan_Dtree.dot','w') as f:
    f=tree.export_graphviz(tree_model,feature_names=features,out_file=f)

Inferences from Classification:

In [19]:
r=tree.export_text(tree_model,feature_names=features)
print(r)

|--- Income <= 113.50
|   |--- CCAvg <= 2.95
|   |   |--- Income <= 106.50
|   |   |   |--- class: 0
|   |   |--- Income >  106.50
|   |   |   |--- Family <= 3.50
|   |   |   |   |--- Education <= 1.50
|   |   |   |   |   |--- class: 0
|   |   |   |   |--- Education >  1.50
|   |   |   |   |   |--- CCAvg <= 1.65
|   |   |   |   |   |   |--- Family <= 1.50
|   |   |   |   |   |   |   |--- CCAvg <= 1.05
|   |   |   |   |   |   |   |   |--- class: 0
|   |   |   |   |   |   |   |--- CCAvg >  1.05
|   |   |   |   |   |   |   |   |--- class: 0
|   |   |   |   |   |   |--- Family >  1.50
|   |   |   |   |   |   |   |--- CCAvg <= 1.00
|   |   |   |   |   |   |   |   |--- class: 1
|   |   |   |   |   |   |   |--- CCAvg >  1.00
|   |   |   |   |   |   |   |   |--- class: 0
|   |   |   |   |   |--- CCAvg >  1.65
|   |   |   |   |   |   |--- Income <= 108.50
|   |   |   |   |   |   |   |--- CCAvg <= 2.58
|   |   |   |   |   |   |   |   |--- class: 0
|   |   |   |   |   |   |   |--- CCAvg >  2.58
|

Some of the inferences:
    
    1) People with Income <= 106.50, CCAvg <= 2.95: Personal Loan :"NO"
            
    2)People with 106.50<Income <= 113.50, Family <= 3.50 , Education <= 1.50, CCAvg <= 2.95- Personal Loan: "No"
        
    3)People with Income >  113.50, Education <= 1.50 and Family <= 2.50: Personal Loan: "No"
            
    4) People with Income >  113.50, Education <= 1.50 and Family > 2.50: Personal Loan: "Yes"..etc
            